# The Model class
*ipcoal* is an object-oriented library for which the main object users interact with is the `Model` class. To create a `Model` you must provide at least one or more parameter arguments to `ipcoal.Model()` which will return an object that you can store as a variable. This stores the information that defines the population demographic model and substitution model, as well as many other options that can modify how simulations are performed. Simulation methods can be called from this object, in addition to several other methods for visualization and analysis.

In [1]:
import ipcoal
import toytree

In [2]:
# initialize a Model object
ipcoal.Model(Ne=1000, nsamples=10)

### Demography (tree)
The demographic structure of a population can be defined in *ipcoal* using the `Model` class argument **tree**. The name of this argument is used to indicate that a tree object can be used to define the parameters of a demographic model. These parameters include the topology (relationships) among populations, the divergence times between populations, and the effective population sizes (Ne) of each population. 

#### A simple Model 
The simplest demographic model is of a single population with constant Ne, which has no topology or divergence times, and only one Ne parameter. This can be created easily be setting the tree argument to its default value of None. As shown in the second example, you will usually initialize a `Model` with many additional arguments, which will be described throughout the rest of this document. 

In [3]:
# a single population model with constant Ne
ipcoal.Model(tree=None, Ne=1000)

In [4]:
# a single population model with constant Ne and other args
ipcoal.Model(Ne=1000, nsamples=5, mut=1e-8, recomb=1-9)

#### Using toytree
An example of a more complex model is a species tree topology with 10 lineages, which requires 9 divergence time parameters and 19 Ne parameters. Setting each of these manually, for example by referring to lineages by an integer label, can be very challenging and error-prone. Instead, the approach we use in *ipcoal* is to create demographic models using `ToyTree` objects, where it is easy to set data values and visualize them to confirm their accuracy. See the [`toytree` library](https://eaton-lab.org/toytree) documentation to learn more about the features for creating and setting data on trees. Here we provide some examples. You can create a tree topology using one of several approaches in `toytree` which return a `ToyTree` class object. This includes the random module (`toytree.rtree`) and the data parsing module (`toytree.tree`). You can further modify a tree by explicitly setting node heights or edge lengths on it, or by using a number of tree modification functions (`toytree.mod`), which can be used to do things like stretch or compress edge lengths.  

In [5]:
# get an imbalanced tree with 3 tips
sptree = toytree.rtree.imbtree(ntips=3)

In [6]:
# or, get a random tree w/ equal spaced nodes and set root height
sptree = toytree.rtree.unittree(ntips=3, treeheight=1e5)

In [7]:
# or, define a custom tree from newick string
sptree = toytree.tree("((A:1,B:1):1,C:2);")

In [8]:
# set the branch lengths on a tree explicitly
sptree = sptree.set_node_data("height", {3: 5e4, 4: 1e5})

In [9]:
# or, set the branch lengths by stretching the tree
sptree = sptree.mod.edges_scale_to_root_height(1e5)

#### Labels and units
Once your species tree relationships and divergence times are set, you can visualize the tree using its `.draw()` function call. The edge lengths of a species tree that will be used to define a demographic model should be **units of generations**. The tips of the tree do not have to align at the present. The tips of the tree will have name labels, whereas internal nodes do not by default, unless set by the user. All nodes have numeric labels which count up from 0 to nnodes-1, in a tips-first then post-order traversal order (see toytree idxorder documentation). The visualization below uses the special tree_style (ts) option type "p" to designate a suite of styling arguments that will show the internal numeric labels on nodes, in addition to the tip labels and the vertical scale bar. Here we are drawing the tree that was created above, which has a root height of 1e5 generations.

In [10]:
# draw the tree to examine the relationships and div times
sptree.draw(ts='p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t312f6eed715e4ab28772a52f14dd2942"> 0 1 2 3 4 A B C 0 50000 100000

#### A complex model
Now that you are sure that the topology and divegence times on your tree are set correctly, you can enter this tree as an argument to initialize a `Model` object with this defined demography. By default, this will sample one gene copy per lineage (tip) of the species tree, but this can be modified (see `nsamples`).

In [11]:
# initialize a Model from a species tree
ipcoal.Model(tree=sptree, Ne=1000)

### Effective population size (Ne)
The effective population size is the key parameter used in population demographic modeling. It determines the probability of coalescence and thus determines the distribution of coalesce times among gene copies in a population. Values of Ne can be set as a constant, or at different values for different populations and ancestral populations in a structured model, such as a species tree. 

#### Ne values are diploid
By default, in *ipcoal* Ne represents the number of **diploid** individuals in a population. Thus, a value of Ne=1000 indicates that there are 2000 haploid gene copies. The coalescent is an approximation of the WrightFisher model, where the probability that any two gene copies share a common ancestor (i.e., coalescence) one generation ago is 1/2N. The expected time until the first coalescent event is therefore 2N. Let's verify this by checking that genealogies simulated in a population with Ne=10K have an average time until first coalescence of 20K generations (i.e., 2 times Ne units of generations).

In [12]:
# simulate many independent trees with 2 samples from a population
NE = 10_000
mod = ipcoal.Model(Ne=NE, nsamples=2, seed_trees=123)
mod.sim_trees(10_000)

# get mean root node height of all trees
trees = [toytree.tree(i) for i in mod.df.genealogy]
avg_time = sum(i.treenode.height for i in trees) / len(trees)

# return result
f"Avg time to first coalescence = {avg_time:.2f} generation, or {avg_time/NE:.2f}Ne"

'Avg time to first coalescence = 20196.12 generation, or 2.02Ne'

#### Setting Ne
When initializing a `Model` object you can set Ne using the `Ne` argument. This accepts a integer, float, dictionary, or None. Setting an integer or float will assign this value to all populations in the model. If a dictionary is entered as {lineage: Ne_value, ...} you can set different Ne values to each lineage, selected by name or tree node index. In this case you must enter a key for every population. 

In [13]:
# set constant Ne to a single population
ipcoal.Model(Ne=1000)

In [14]:
# set constant Ne to all Nodes on a species tree
ipcoal.Model(tree=sptree, Ne=1000)

In [15]:
# set variable Ne values to Nodes on a species tree
ipcoal.Model(tree=sptree, Ne={0: 1000, 1: 1000, 2: 1000, 3: 2000, 4: 2000})

#### Setting Ne on trees
Finally, another way to set up complex demographic models is to set all data explicitly on the tree object. To use this option you must enter the Ne argument to `ipcoal.Model` as None. Ne values will then be automatically extracted from the tree input, if present. An error will be raised if all nodes do not have an Ne values set. Note that if an Ne argument (besides None) is entered to `ipcoal.Model` it will override any Ne values set on a tree. To set Ne data on a tree object we use the `set_node_data` function in `toytree`, which can be used to set any abitrary named feature on nodes of a tree. Here the feature name "Ne" is special, as *ipcoal* will look for and recognize it if present. This approach can be more convenient than entering Ne values as a dictionary, like above, because the `set_node_data` function has some helpful features such as setting a default value to nodes that are not specified, and because the values set to nodes can be easily visualized on trees.

In [16]:
# set variable Ne to each Node
sptree = sptree.set_node_data("Ne", {0: 1e4, 1: 2e4, 2: 3e4, 3: 5e4, 4: 1e5})

In [17]:
# set variable Ne to some Nodes and others to a default
sptree = sptree.set_node_data("Ne", {3: 5e4, 4: 1e5}, default=1e4)

In [18]:
# visualize Ne as edge widths projected between size 2-10
sptree.draw('p', edge_widths=("Ne", 2, 10));

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5e0819f711be45419eacc5e23d1037c4"> 0 1 2 3 4 A B C 0 50000 100000

In [19]:
# finally, we can enter the tree to set Ne values in the Model
ipcoal.Model(tree=sptree)

#### Variable Ne within intervals
*ipcoal* is not currently designed to implement more complex scenarios, such as Ne values that change linearly, or exponentially, from the beginning to the end of an interval. These should be simple enough to implement, but our focus thus far has been primarily on species tree scenarios which have constant Ne within each interval. To set up more complex demographic models I would recommend using `msprime` for now. If you are interested in helping to implement this please reach out.

### Sampling (nsamples)
A key feature of the coalescent model, which involves studying the process of evolution backwards in time, is that we do not need to simulate the history of an entire population in order to study its evolution. Instead, we can examine only the history for a set of sampled gene copies in that population. In this way, the *number of samples* plays an important role in many characteristics of the coalescent. This can be set using the `nsamples` argument to `ipcoal.Model`. This can be entered as an integer, which sets the number of gene copies to sample from every population, or as a dict, where different numbers of gene copies can be sampled from each population. Here, a sample represents a haploid gene copy. Note that it is still possible to simulate diploid sequences in *ipcoal* by sampling an even number of samples in each population. The implementation of joining pairs of samples into diploids only takes place within downstream analyses, such as writing sequences. To demonstrate the `nsamples` argument we show several examples of initialized `Model` objects below, where we show the imap dictionary of each, which maps population names to lists of sample names. 

In [20]:
ipcoal.Model(Ne=1000, nsamples=5).get_imap_dict()

{'p': ['p_0', 'p_1', 'p_2', 'p_3', 'p_4']}

In [21]:
ipcoal.Model(sptree, nsamples=2).get_imap_dict()

{'A': ['A_0', 'A_1'], 'B': ['B_0', 'B_1'], 'C': ['C_0', 'C_1']}

In [22]:
ipcoal.Model(sptree, nsamples={"A": 1, "B": 2, "C": 3}).get_imap_dict()

{'A': ['A_0'], 'B': ['B_0', 'B_1'], 'C': ['C_0', 'C_1', 'C_2']}

Finally, using visualizations we can validate the setting for different numbers of samples in each population.

In [25]:
mod = ipcoal.Model(sptree, nsamples={"A": 1, "B": 2, "C": 3})
mod.sim_trees(1)
mod.draw_demography(0);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t653251b516b04d05aa8ebf4f2fb7b029"> idx=3
Ne=5e+04
Tc=0.5
Tg=5e+04 idx=2
Ne=1e+04
Tc=5
Tg=1e+05 idx=0
Ne=1e+04
Tc=2.5
Tg=5e+04 idx=1
Ne=1e+04
Tc=2.5
Tg=5e+04 idx=4
Ne=1e+05
Tc=0
Tg=0 A_0 B_0 B_1 C_2 C_0 C_1 node=0 node=1 node=2 node=6 node=3 node=4 node=5 node=7 node=8 node=9 node=10 0 100000 200000 300000

### Gene flow (admixture_edges)
In both *ipcoal* and *toytree* you define an admixture event with one or more 4-part tuples. This designates the (source, destination, time, rate/prop). If the time is a single value then an admixture event occurs by transferring a *proportion* of one population to another; whereas if time is entered as a tuple of (time_min, time_max) it occurs as a migration *rate* over this period of time. In simulations you will need to provide all four arguments, but for tree drawings you only need to provide the first two and the latter two will be automated to make it look nice. So let’s try first to draw an admixture edge on a tree.

This is done by supplying the event tuple to the `admixture_edges` tree drawing argument. Here the source and destination refer to the index (idx) numeric labels of each node (which you find by drawing the tree). You can see that the admixture edge is not simply a horizontal line, but it follows the edges towards the tips of the source node and towards the ancestors from the destination node. This is to make the direction of inheritance of the introgression clear. In this case, a tuple of (1, 2) indicates that introgression occurs from 1 to 2 backwards in time; meaning that some alleles that arose in the ancestor of node 2 have introgressed into lineage 1 at the present.

In [26]:
# draw the admixture_edges scenario to validate its set up
sptree.draw(ts='p', admixture_edges=[(1, 2)]);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t91a9c8b67fd54e28a2cfc37478542999"> 0 1 2 3 4 A B C 0 50000 100000

#### Admixture
Now let’s define an admixture event to a `Model` object. The proportion here is set to 0.1 (10% admixture). The timing of the event can be entered either as a integer, representing the time in generations at which it occurs, or as a float, in which case the timing will be modeled as occuring at the set percentage of the way back in time along the time at which the two edges both existed. The latter argument is useful for some automation scenarios so you do not need to find when two edges coexisted in time. Both are demonstrated below. 

In [27]:
# admixture of 10% from 1->2 at time=2000
amodel = ipcoal.Model(sptree, Ne=1e4, admixture_edges=[(1, 2, 2000, 0.1)], seed_trees=123)

# simulate 100 unlinked genealogies under this model
amodel.sim_trees(100)

# visualize discordance as a cloud tree
amodel.draw_cloud_tree(width=300, scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0f1eff469a884233b71e36ce11790096"> C A B 0 100000 200000

Here is a similar scenario but we have set the timing of the introgression to occur further back in time, at 50% of the way back along the interval during which edges 1 and 2 both exist. 

In [28]:
# admixture of 10% from 1->2 at time=50% of way back on shared history
amodel = ipcoal.Model(sptree, Ne=1e4, admixture_edges=[(1, 2, 0.5, 0.1)], seed_trees=123)

# simulate 100 unlinked genealogies under this model
amodel.sim_trees(100)

# visualize discordance as a cloud tree
amodel.draw_cloud_tree(width=300, scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf6c06d11cc7146a58d27dc623caea8ef"> C A B 0 100000 200000

#### Migration
Next is an example where we model a *migration* event instead of an *admixture* event. The difference is that instead of an instantaneous mixing event, we model a migration rate from one population to another over a period of time. Similar to above, the period of time can be set as a distinct start and end time as integer generations, or it can be set as a float to indicate proportion of time along their shared time interval. Both are demonstrated below.

In [29]:
# migration at rate 1e-4 from 1->2 from time 0-1000
amodel = ipcoal.Model(sptree, Ne=1e4, admixture_edges=[(1, 2, (0, 1000), 1e-4)], seed_trees=123)

# simulate 100 unlinked genealogies under this model
amodel.sim_trees(100)

# visualize discordance as a cloud tree
amodel.draw_cloud_tree(width=250, scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="250.0px" height="300.0px" viewBox="0 0 250.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0f056eba80794ba0bdd907f65d26c14b"> C A B 0 100000 200000

In [30]:
# migration at rate 2e-5 from 1->2 from time 0%-50% of their time 
amodel = ipcoal.Model(sptree, Ne=1e4, admixture_edges=[(1, 2, (0.0, 0.5), 2e-5)], seed_trees=123)

# simulate 100 unlinked genealogies under this model
amodel.sim_trees(100)

# visualize discordance as a cloud tree
amodel.draw_cloud_tree(scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t83babcd2de244ca28a649e7bc6d83695"> C A B 0 100000 200000

Finally, you can set more than one migration or admixture event at a time by simply entering them as multiple tuples. In this example we set bi-directional migration between B and C. 

In [31]:
# migration at rate 1e-5 from 1<->2 from time 0%-50% of their time 
amodel = ipcoal.Model(
    sptree, Ne=1e4, seed_trees=123,
    admixture_edges=[
        (1, 2, (0.0, 0.5), 1e-5),
        (2, 1, (0.0, 0.5), 1e-5),
    ])

# simulate 100 unlinked genealogies under this model
amodel.sim_trees(100)

# visualize discordance as a cloud tree
c, a, m = amodel.draw_cloud_tree(scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taf33e89c4fa44fb7b94ff67ba453bf23"> C A B 0 50000 100000 150000 200000

The effective population size plays enormous role in demographic modeling, where in the absence of natural selection or gene flow, this parameter alone can describe the expected distribution of coalescent times in a population. Given the same number of sampled gene copies in a population, larger Ne values will lead to deeper and more variable coalescent times, whereas smaller Ne will lead to smaller and less variable coalescent times.

In [29]:
# set up a low and high Ne model
model1 = ipcoal.Model(sptree, Ne=1e4)
model2 = ipcoal.Model(sptree, Ne=1e6)

# simulate 10 unlinked genealogies on each
model1.sim_trees(10)
model2.sim_trees(10)

In [30]:
# draw the low Ne unlinked genealogies
c, a, m = model1.draw_genealogies(range(5), shape=(1, 5), shared_axes=True, scale_bar=1e4);
c.text(350, 20, "<b>Low Ne genealogies</b>")

# draw the high Ne unlinked genealogies
c, a, m = model2.draw_genealogies(range(5), shape=(1, 5), shared_axes=True, scale_bar=1e4);
c.text(350, 20, "<b>High Ne genealogies</b>");

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="250.0px" viewBox="0 0 750.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t63fba273e4884063b396c97f9f3b462d"> C A B 0 5 10 C A B 0 5 10 C A B 0 5 10 C A B 0 5 10 C A B 0 5 10 Low Ne genealogies

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="250.0px" viewBox="0 0 750.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta683049272964611b9fb25d26d485905"> A B C 0 100 200 C A B 0 100 200 B A C 0 100 200 C A B 0 100 200 C A B 0 100 200 High Ne genealogies